In [1]:
import torch
from transformers import OPTForCausalLM, AutoModelForCausalLM, AutoTokenizer
# from chemlactica.utils.utils import get_tokenizer
from rdkit.Chem import Descriptors

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import RDConfig, MACCSkeys, QED
import matplotlib.pyplot as plt
import numpy as np
import json
import pickle
from sklearn import metrics
from scipy.stats import spearmanr
from rdkit.Chem.rdMolDescriptors import CalcTPSA
from rdkit import Chem, DataStructs
import sys
import os
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer

/auto/home/menuab/miniforge3/envs/llama3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd    
jsonObj = pd.read_json(path_or_buf="/nfs/ap/mnt/sxtn/rdkit_computed_rel+form/valid_rdkit_computed_rel+form/950001_start.jsonl", lines=True)

In [3]:
smiles = list(jsonObj[jsonObj!=""].dropna().SMILES.values)

In [4]:
len(smiles)

10927

In [5]:
np.random.seed(42)
smiles100 = np.random.choice(smiles, size=100)

In [6]:
smiles100

array(['CC(=O)Nc1ccc(C(=O)NN=Cc2cccc([N+](=O)[O-])c2OC(=O)c2cccc(C)c2)cc1',
       'CCNC(=NCC(C)Cc1cccs1)NC1CCN(c2ncccc2F)C1',
       'CCOC(=O)C1C(C)NNC1S(=O)(=O)Nc1cccc(CC)c1',
       'CCc1n[nH]c2c(c1=S)CN(C(=O)c1ccccc1)CC2',
       'CNc1cccc(Nc2cnn(C)c2)c1[N+](=O)[O-]',
       'O=C(NCC(O)CN1CCOCC1)Nc1snc(OCc2ccc(Cl)cc2F)c1C(=O)O',
       'CCC(=O)c1sc(NC2C3C4CCC(C4)C23)c(OC)c1N',
       'CN(C)S(=O)(=O)NCCCc1ncc[nH]1',
       'Cc1ccc(F)c(OCc2cc(F)cc(C#CCO)c2)c1',
       'Cc1cc(C)c(S(=O)(=O)NCC2CCCN2c2ccccc2)c(C)c1',
       'Cc1nn(C)c(Oc2ccc(Br)cc2[N+](=O)[O-])c1CO',
       'CN=C(NCc1nc(-c2ccccc2)cs1)N1CCSC(C)(C)C1',
       'CC(C)NS(=O)(=O)[N-]Cc1ccccc1.O=C(O)c1cccc2c1cnn2-c1ccc(F)cc1',
       'CCOc1ccc(-c2noc(Cn3c(=O)n(C(C)C)c(=O)c4ccccc43)n2)cc1OC',
       'O=C(CSc1nnnn1-c1ccc(Cl)cc1)Nc1cccc(Cl)c1N1CCCCC1',
       'Cc1ccccc1C1(O)CCN(C(=O)c2ccc(-c3ccc(F)cc3)o2)CC1',
       'CCc1ccc(NC(=O)NC(C)(C)CCC(=O)O)cc1S(N)(=O)=O',
       'CCCOc1ccc(C=NNC(=O)COc2ccccc2C)cc1OC',
       'CC(C)(C)OC(

In [7]:
pubchem_stats_file = open("/auto/home/menuab/code/ChemLacticaTestSuite/src/stats_data/pubchem_stats.pkl", 'rb')
pubchem_stats = pickle.load(pubchem_stats_file)
pubchem_stats_file.close()

In [8]:
llama_tokenizer_path = "/nfs/h100/raid/chem/checkpoints/tokenizers/Llama-3.2-chem-1B-v1"
model_1b_20k_safe_273f = "/nfs/h100/raid/chem/checkpoints/hf/yerevann/Llama-3.2-1B/273fe38764a44b0db611e262/step-20000"
model_1b_20k_8a28 = "/nfs/h100/raid/chem/checkpoints/hf/yerevann/Llama-3.2-1B/8a28eb82260140b097b34e1c/step-20000"

In [9]:
# tokenizer = AutoTokenizer.from_pretrained("/auto/home/menuab/code/ChemLactica/chemlactica/tokenizer/ChemLacticaTokenizer66")
# tokenizer = AutoTokenizer.from_pretrained("/auto/home/menuab/code/ChemLactica/chemlactica/tokenizer/GemmaTokenizer")
# tokenizer = AutoTokenizer.from_pretrained("yerevann/chemlactica-125m")
tokenizer = AutoTokenizer.from_pretrained(llama_tokenizer_path)
tokenizer.add_special_tokens = False
tokenizer.pad_token_id = tokenizer.eos_token_id
len(tokenizer)

128330

In [10]:
model_125m_20k_9954 = "/nfs/dgx/raid/chem/checkpoints/facebook/galactica-125m/9954e52e400b43d18d3a40f6/checkpoint-20480"
model_2b_11k_d6e6 = "/nfs/dgx/raid/chem/checkpoints/google/gemma-2b/d6e6a76e91814ad68d5fa264/checkpoint-11000"
model_2b_2k_699e = "/nfs/dgx/raid/chem/checkpoints/google/gemma-2b/699e8c6078bb4461a73b39de/checkpoint-2000"
model_125m_18k_1f28 = "/nfs/dgx/raid/chem/checkpoints/facebook/galactica-125m/1f289ff103034364bd27e1c3/checkpoint-18000/"
model_2b_12k_0717 = "/nfs/dgx/raid/chem/checkpoints/h100/google/gemma-2b/0717d445bcf44e31b2887892/checkpoint-12000"
model_2b_18k_0717 = "/nfs/dgx/raid/chem/checkpoints/h100/google/gemma-2b/0717d445bcf44e31b2887892/checkpoint-18000"
model_1b_19k_6d68 = "/nfs/dgx/raid/chem/checkpoints/h100/facebook/galactica-1.3b/6d68b252d53647a99cf2fa8b/checkpoint-19000"
model_125m_18k_c4d8 = "/nfs/dgx/raid/chem/titan_outputs/hf/yerevann/chemlactica-125m"
model_125m_60k_0bb9 = "/nfs/dgx/raid/chem/random_init_exp/checkpoints/facebook/galactica-125m/0bb9077eb9824a30a082f020/checkpoint-60000"
model_125m_80k_0bb9 = "/nfs/dgx/raid/chem/random_init_exp/checkpoints/facebook/galactica-125m/0bb9077eb9824a30a082f020/checkpoint-80000"
model_125m_45k_0bb9 = "/nfs/dgx/raid/chem/random_init_exp/checkpoints/facebook/galactica-125m/0bb9077eb9824a30a082f020/checkpoint-45000"

# model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16).eval()
# model.device, model.dtype

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    model_1b_20k_8a28,
).to("cuda:0").eval()
model.device, model.dtype

(device(type='cuda', index=0), torch.float32)

In [12]:
def calculate_tanim_sim(m, rel_m):
    m=Chem.MolFromSmiles(m)
    rel_m =Chem.MolFromSmiles(rel_m)
    # AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    fp = AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=2048)
    rel_fp = AllChem.GetMorganFingerprintAsBitVect(rel_m, 2, nBits=2048)
    return DataStructs.TanimotoSimilarity(fp, rel_fp)

In [13]:
def generate_plot(target_clean, generated_clean, diffs, test_name, rmse, mape, correlation, n_invalid_generations, n_total_gens, property_range, thickness):
    max_, min_, max_g = np.max(target_clean), np.min(target_clean), np.max(generated_clean)
    # title = f'model_1b_19k_6d68 {test_name} Greedy sampling\n'\
    #         f'{n_invalid_generations}/{n_total_gens} invalid SMILES\n'\
    #         f'rmse {rmse:.3f} mape {mape:.3f} corr: {correlation:.3f}\n'\
    if test_name == "Similarity CG":
        title = f'Similarity Conditional Generation (greedy sampling)\n'\
                f'{n_invalid_generations}/{n_total_gens} invalid SMILES\n'\
                f'rmse {rmse:.3f} mape {mape:.3f} corr: {correlation:.3f}\n'
    else:
        title = f'{test_name} Property Prediction (greedy sampling)\n'\
                f'{n_invalid_generations}/{n_total_gens} invalid SMILES\n'\
                f'rmse {rmse:.3f} mape {mape:.3f} corr: {correlation:.3f}\n'

    fig, ax1 = plt.subplots()
    fig.set_figheight(4)
    fig.set_figwidth(5)
    fig.set_linewidth(2)
    if thickness != 0:    
        ax2 = ax1.twinx()
        stats = pubchem_stats[test_name.upper()]
    #     property_range = [1, 10]
        # stats_width = (property_range[1] - property_range[0]) / thickness
        ax2.bar([interval.mid for interval in stats.index], stats, width=thickness, alpha=0.3) 
    dist = max_ - min_
    margin = 0.05
    ax1.set_xlim([min_- margin*dist, max_ + margin*dist])
    ax1.scatter(target_clean, generated_clean, c='b')
    # ax1.vlines(nones, ymin=min_, ymax=max_, color='r', alpha=0.3)
    ax1.plot([min_, max_], [min_, max_], color='grey', linestyle='--', linewidth=2)
    if test_name == "Similarity CG":
        ax1.plot(np.arange(0.2,1.05,0.05), np.convolve(diffs, np.ones(3)/3, mode='same'), color='m', alpha=0.5)
        ax1.set_xlabel(f'Target Similarity')
        ax1.set_ylabel(f'Generated Similarity')
    else:
        ax1.plot(target_clean, np.convolve(diffs, np.ones(5)/5, mode='same'), color='m', alpha=0.5)
        ax1.set_xlabel(f'Ground truth {test_name}')
        ax1.set_ylabel(f'Predicted {test_name}')
    ax1.grid(True)
    plt.title(title)
    plt.tight_layout()
    fig.savefig(f'{test_name}_property.png', dpi=300, format="png")
    # fig.savefig(f'{test_name}_property.pdf', dpi=300, format="pdf")
    fig.clf()
    plt.close()

In [15]:
tic = {
        "QED": .01,
        "SAS": .1,
        "TPSA": 1,
        "WEIGHT": 12,
        "CLOGP": .25,
        }
properties = ["TPSA","WEIGHT","QED","SAS","CLOGP"]
for property in properties:
    ground_truths, gens, diffs = [],[],[]
    invalids = 0
    for s in smiles100:
        prompt = f"<|begin_of_text|>[SMILES]{s}[/SMILES][{property}]"
        len_prompt = len(prompt)
        prompt = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)
        out = model.generate(prompt.input_ids, do_sample=False, eos_token_id=tokenizer.encode(f"[/{property}]", add_special_tokens=False)[0], max_new_tokens=300)
        out = tokenizer.batch_decode(out)[0]
        try:
            if out.find(f"[{property}]")!=-1:
                gen_score = float(out[out.find(f"[{property}]") + len(f"[{property}]"):out.find(f"[/{property}]")])
                if property == "TPSA":
                    gt_score = AllChem.CalcTPSA(Chem.MolFromSmiles(s))
                elif property == "QED":
                    gt_score = QED.qed(Chem.MolFromSmiles(s))
                elif property == "SAS":
                    gt_score = sascorer.calculateScore(Chem.MolFromSmiles(s))
                elif property == "WEIGHT":
                    gt_score = Descriptors.ExactMolWt(Chem.MolFromSmiles(s))
                elif property == "CLOGP":
                    gt_score = Descriptors.MolLogP(Chem.MolFromSmiles(s))
                diff = abs(gt_score - gen_score)
                print("GT:", round(gt_score,2), "Gen:", gen_score, "diff:", round(diff,2), s, out )
                ground_truths.append(gt_score)
                gens.append(gen_score)
                diffs.append(diff)
            else:
                print(f"GT: {gt_score} GEN: {gen_score} {out}")
        except:
            print(f"GT: {gt_score} {out}")
            invalids += 1
            raise

    combined = list(zip(ground_truths, gens, diffs))
    combined.sort(key=lambda x: x[0])
    ground_truths, gens, diffs = zip(*combined)
    rmse = metrics.mean_squared_error(ground_truths, gens, squared=False)
    mape = metrics.mean_absolute_percentage_error(ground_truths, gens)
    correlation = correlation, pvalue = spearmanr(ground_truths, gens)
    # tic = 900 if property in ["TPSA", "WEIGHT"] else 100
    generate_plot(ground_truths, gens, diffs, property, rmse, mape, correlation, invalids, len(ground_truths), (min(ground_truths), max(ground_truths)), tic[property])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GT: 140.0 Gen: 140.0 diff: 0.0 CC(=O)Nc1ccc(C(=O)NN=Cc2cccc([N+](=O)[O-])c2OC(=O)c2cccc(C)c2)cc1 <|begin_of_text|>[SMILES]CC(=O)Nc1ccc(C(=O)NN=Cc2cccc([N+](=O)[O-])c2OC(=O)c2cccc(C)c2)cc1[/SMILES][TPSA]140.00[/TPSA]
GT: 52.55 Gen: 52.55 diff: 0.0 CCNC(=NCC(C)Cc1cccs1)NC1CCN(c2ncccc2F)C1 <|begin_of_text|>[SMILES]CCNC(=NCC(C)Cc1cccs1)NC1CCN(c2ncccc2F)C1[/SMILES][TPSA]52.55[/TPSA]
GT: 96.53 Gen: 96.53 diff: 0.0 CCOC(=O)C1C(C)NNC1S(=O)(=O)Nc1cccc(CC)c1 <|begin_of_text|>[SMILES]CCOC(=O)C1C(C)NNC1S(=O)(=O)Nc1cccc(CC)c1[/SMILES][TPSA]96.53[/TPSA]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


GT: 48.99 Gen: 48.99 diff: 0.0 CCc1n[nH]c2c(c1=S)CN(C(=O)c1ccccc1)CC2 <|begin_of_text|>[SMILES]CCc1n[nH]c2c(c1=S)CN(C(=O)c1ccccc1)CC2[/SMILES][TPSA]48.99[/TPSA]
GT: 85.02 Gen: 85.02 diff: 0.0 CNc1cccc(Nc2cnn(C)c2)c1[N+](=O)[O-] <|begin_of_text|>[SMILES]CNc1cccc(Nc2cnn(C)c2)c1[N+](=O)[O-][/SMILES][TPSA]85.02[/TPSA]
GT: 133.25 Gen: 133.25 diff: 0.0 O=C(NCC(O)CN1CCOCC1)Nc1snc(OCc2ccc(Cl)cc2F)c1C(=O)O <|begin_of_text|>[SMILES]O=C(NCC(O)CN1CCOCC1)Nc1snc(OCc2ccc(Cl)cc2F)c1C(=O)O[/SMILES][TPSA]133.25[/TPSA]


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GT: 6.34 Gen: 6.0 diff: 0.34 CC(=O)OCC1=CC2(O)OC1CC1C3=C(C(C)C)CCC3(C)CCC12C.CC(C)C(=O)CCC1(C)CCC2(C)C(CC=C3C(O)C4(O)COC5OC2C3C5(O)C4O)C1=O.CC(C)C(=O)CCC1(C)CCC2(C)C(CCC3C(O)OC4(O)C(O)COC5OC2C3C54O)C1=O.CC(C)C1=C2C3=CC=C(C(=O)[O-])CC(OC4OCC(O)C(O)C4O)C3(C)CCC2(C)CC1.CC(C)C1=C2C3CC=C4C(O)C5(O)COC6OC(C4C6(O)C5O)C3(C)CCC2(C)CC1.[Na+] <|begin_of_text|>[SMILES]CC(=O)OCC1=CC2(O)OC1CC1C3=C(C(C)C)CCC3(C)CCC12C.CC(C)C(=O)CCC1(C)CCC2(C)C(CC=C3C(O)C4(O)COC5OC2C3C5(O)C4O)C1=O.CC(C)C(=O)CCC1(C)CCC2(C)C(CCC3C(O)OC4(O)C(O)COC5OC2C3C54O)C1=O.CC(C)C1=C2C3=CC=C(C(=O)[O-])CC(OC4OCC(O)C(O)C4O)C3(C)CCC2(C)CC1.CC(C)C1=C2C3CC=C4C(O)C5(O)COC6OC(C4C6(O)C5O)C3(C)CCC2(C)CC1.[Na+][/SMILES][CLOGP]6.00[/CLOGP]
GT: 6.49 Gen: 6.41 diff: 0.08 CCOc1cc(C2C(C(=O)Nc3ccc(Cl)cc3)=C(C)Nc3ncnn32)ccc1OCc1ccccc1Cl <|begin_of_text|>[SMILES]CCOc1cc(C2C(C(=O)Nc3ccc(Cl)cc3)=C(C)Nc3ncnn32)ccc1OCc1ccccc1Cl[/SMILES][CLOGP]6.41[/CLOGP]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GT: 7.54 Gen: 7.54 diff: 0.0 CC1CCCC(C)C1n1c(-c2cccs2)nc2cc(C(=O)NC3(C(=O)Nc4ccc(C=CC(=O)O)cc4)CCCC3)ccc21 <|begin_of_text|>[SMILES]CC1CCCC(C)C1n1c(-c2cccs2)nc2cc(C(=O)NC3(C(=O)Nc4ccc(C=CC(=O)O)cc4)CCCC3)ccc21[/SMILES][CLOGP]7.54[/CLOGP]
GT: 2.88 Gen: 2.9 diff: 0.02 CCC(CC)n1nc(C)c(C(C)N)c1C <|begin_of_text|>[SMILES]CCC(CC)n1nc(C)c(C(C)N)c1C[/SMILES][CLOGP]2.90[/CLOGP]
GT: 3.69 Gen: 3.6 diff: 0.09 Nc1nc(C(F)(F)F)nn1-c1cc(S)c(C2CC2)cc1Cl <|begin_of_text|>[SMILES]Nc1nc(C(F)(F)F)nn1-c1cc(S)c(C2CC2)cc1Cl[/SMILES][CLOGP]3.60[/CLOGP]


/auto/home/menuab/miniforge3/envs/llama3/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [16]:
ground_truths, gens, diffs = [],[],[]
invalids = 0
for s in smiles100[:10]:
    for s2 in smiles100[10:20]:
        prompt = f"<|begin_of_text|>[SMILES]{s}[/SMILES][SIMILAR]{s2} "
        len_prompt = len(s2) + 1
        prompt = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)
        out = model.generate(prompt.input_ids, do_sample=False, eos_token_id=tokenizer.encode("[/SIMILAR]", add_special_tokens=False)[0], max_new_tokens=300)
        out = tokenizer.batch_decode(out)[0]
        try:
            if out.find("[/SIMILAR]")!=-1:
                gen_score = round(float(out[out.find("[SIMILAR]") + len("[SIMILAR]") + len_prompt:out.find("[/SIMILAR]")]), 2)
                # tpsa_score = AllChem.CalcTPSA(Chem.MolFromSmiles(smiles))
                # gt_score = round(QED.qed(Chem.MolFromSmiles(s)), 2)
                # gt_score = round(sascorer.calculateScore(Chem.MolFromSmiles(s)), 2)
                gt_score = calculate_tanim_sim(s, s2)
                diff = abs(gt_score - gen_score)
                print("GT:", round(gt_score,2), "Gen:", gen_score, "diff:", round(diff,2), s, out )
                ground_truths.append(gt_score)
                gens.append(gen_score)
                diffs.append(diff)
            else:
                print(f"GT: {gt_score} GEN: {gen_score} {out}")
        except:
            print(f"\n*********s: {s}, s2 {s2}, {out}")
            invalids += 1
            raise

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
[16:07:53] DEPRECATION WARNING: please use MorganGenerator
[16:07:53] DEPRECATION WARNING: please use MorganGenerator
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
[16:07:53] DEPRECATION WARNING: please use MorganGenerator
[16:07:53] DEPRECATION WARNING: please use MorganGenerator
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GT: 0.2 Gen: 0.2 diff: 0.0 CC(=O)Nc1ccc(C(=O)NN=Cc2cccc([N+](=O)[O-])c2OC(=O)c2cccc(C)c2)cc1 <|begin_of_text|>[SMILES]CC(=O)Nc1ccc(C(=O)NN=Cc2cccc([N+](=O)[O-])c2OC(=O)c2cccc(C)c2)cc1[/SMILES][SIMILAR]Cc1nn(C)c(Oc2ccc(Br)cc2[N+](=O)[O-])c1CO 0.20[/SIMILAR]
GT: 0.09 Gen: 0.13 diff: 0.04 CC(=O)Nc1ccc(C(=O)NN=Cc2cccc([N+](=O)[O-])c2OC(=O)c2cccc(C)c2)cc1 <|begin_of_text|>[SMILES]CC(=O)Nc1ccc(C(=O)NN=Cc2cccc([N+](=O)[O-])c2OC(=O)c2cccc(C)c2)cc1[/SMILES][SIMILAR]CN=C(NCc1nc(-c2ccccc2)cs1)N1CCSC(C)(C)C1 0.13[/SIMILAR]


[16:07:54] DEPRECATION WARNING: please use MorganGenerator
[16:07:54] DEPRECATION WARNING: please use MorganGenerator
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
[16:07:54] DEPRECATION WARNING: please use MorganGenerator
[16:07:54] DEPRECATION WARNING: please use MorganGenerator
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GT: 0.12 Gen: 0.24 diff: 0.12 CC(=O)Nc1ccc(C(=O)NN=Cc2cccc([N+](=O)[O-])c2OC(=O)c2cccc(C)c2)cc1 <|begin_of_text|>[SMILES]CC(=O)Nc1ccc(C(=O)NN=Cc2cccc([N+](=O)[O-])c2OC(=O)c2cccc(C)c2)cc1[/SMILES][SIMILAR]CC(C)NS(=O)(=O)[N-]Cc1ccccc1.O=C(O)c1cccc2c1cnn2-c1ccc(F)cc1 0.24[/SIMILAR]
GT: 0.09 Gen: 0.17 diff: 0.08 CC(=O)Nc1ccc(C(=O)NN=Cc2cccc([N+](=O)[O-])c2OC(=O)c2cccc(C)c2)cc1 <|begin_of_text|>[SMILES]CC(=O)Nc1ccc(C(=O)NN=Cc2cccc([N+](=O)[O-])c2OC(=O)c2cccc(C)c2)cc1[/SMILES][SIMILAR]CCOc1ccc(-c2noc(Cn3c(=O)n(C(C)C)c(=O)c4ccccc43)n2)cc1OC 0.17[/SIMILAR]


[16:07:54] DEPRECATION WARNING: please use MorganGenerator
[16:07:54] DEPRECATION WARNING: please use MorganGenerator
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
[16:07:54] DEPRECATION WARNING: please use MorganGenerator
[16:07:54] DEPRECATION WARNING: please use MorganGenerator
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

[16:08:03] DEPRECATION WARNING: please use MorganGenerator
[16:08:03] DEPRECATION WARNING: please use MorganGenerator
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
[16:08:04] DEPRECATION WARNING: please use MorganGenerator
[16:08:04] DEPRECATION WARNING: please use MorganGenerator
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GT: 0.17 Gen: 0.22 diff: 0.05 Cc1cc(C)c(S(=O)(=O)NCC2CCCN2c2ccccc2)c(C)c1 <|begin_of_text|>[SMILES]Cc1cc(C)c(S(=O)(=O)NCC2CCCN2c2ccccc2)c(C)c1[/SMILES][SIMILAR]Cc1ccccc1C1(O)CCN(C(=O)c2ccc(-c3ccc(F)cc3)o2)CC1 0.22[/SIMILAR]
GT: 0.15 Gen: 0.17 diff: 0.02 Cc1cc(C)c(S(=O)(=O)NCC2CCCN2c2ccccc2)c(C)c1 <|begin_of_text|>[SMILES]Cc1cc(C)c(S(=O)(=O)NCC2CCCN2c2ccccc2)c(C)c1[/SMILES][SIMILAR]CCc1ccc(NC(=O)NC(C)(C)CCC(=O)O)cc1S(N)(=O)=O 0.17[/SIMILAR]


[16:08:04] DEPRECATION WARNING: please use MorganGenerator
[16:08:04] DEPRECATION WARNING: please use MorganGenerator
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
[16:08:04] DEPRECATION WARNING: please use MorganGenerator
[16:08:04] DEPRECATION WARNING: please use MorganGenerator
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GT: 0.15 Gen: 0.23 diff: 0.08 Cc1cc(C)c(S(=O)(=O)NCC2CCCN2c2ccccc2)c(C)c1 <|begin_of_text|>[SMILES]Cc1cc(C)c(S(=O)(=O)NCC2CCCN2c2ccccc2)c(C)c1[/SMILES][SIMILAR]CCCOc1ccc(C=NNC(=O)COc2ccccc2C)cc1OC 0.23[/SIMILAR]
GT: 0.16 Gen: 0.22 diff: 0.06 Cc1cc(C)c(S(=O)(=O)NCC2CCCN2c2ccccc2)c(C)c1 <|begin_of_text|>[SMILES]Cc1cc(C)c(S(=O)(=O)NCC2CCCN2c2ccccc2)c(C)c1[/SMILES][SIMILAR]CC(C)(C)OC(=O)NCC(=O)N1CCOC(c2ccccc2F)C1 0.22[/SIMILAR]
GT: 0.13 Gen: 0.16 diff: 0.03 Cc1cc(C)c(S(=O)(=O)NCC2CCCN2c2ccccc2)c(C)c1 <|begin_of_text|>[SMILES]Cc1cc(C)c(S(=O)(=O)NCC2CCCN2c2ccccc2)c(C)c1[/SMILES][SIMILAR]CCOc1cc(OC)c(Br)cc1CC1CCCNC1 0.16[/SIMILAR]


[16:08:04] DEPRECATION WARNING: please use MorganGenerator
[16:08:04] DEPRECATION WARNING: please use MorganGenerator


In [17]:
combined = list(zip(ground_truths, gens, diffs))
combined.sort(key=lambda x: x[0])
ground_truths, gens, diffs = zip(*combined)
rmse = metrics.mean_squared_error(ground_truths, gens, squared=False)
mape = metrics.mean_absolute_percentage_error(ground_truths, gens)
correlation = correlation, pvalue = spearmanr(ground_truths, gens)
generate_plot(ground_truths, gens,diffs, 'Similarity', rmse, mape, correlation, invalids, len(ground_truths), (0,1), 0)

/auto/home/menuab/miniforge3/envs/llama3/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [20]:
ground_truths, gens, diffs = [],[],[]
invalids = 0
N_mols = 20
for s in smiles100[:N_mols]:
    for sim in np.arange(0.2,1.05,0.05):
        prompt = f"<|begin_of_text|>[SIMILAR]{s} {sim:.2f}[/SIMILAR]"
        prompt = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(model.device)
        out = model.generate(prompt.input_ids, do_sample=False, eos_token_id=tokenizer.encode("[/SMILES]", add_special_tokens=False)[0],
        # suppress_tokens=[2, 10, 44, 11, 45, 12, 46, 13, 47],
        # suppress_tokens=[2, 50002, 50035, 50003, 50036, 50000, 50033],
        suppress_tokens=[34277, 128260, 128261, 128262, 128263, 128264, 128265, 128266, 128267],
          repetition_penalty=1, renormalize_logits=True,max_length=2000)
        out = tokenizer.batch_decode(out)[0]
        if out.find("[/SMILES]")!=-1:
            try:
                mol = out[out.find("[SMILES]") + len("[SMILES]"):out.find("[/SMILES]")]
                # tpsa_score = AllChem.CalcTPSA(Chem.MolFromSmiles(smiles))
                # gt_score = round(QED.qed(Chem.MolFromSmiles(s)), 2)
                # gt_score = round(sascorer.calculateScore(Chem.MolFromSmiles(s)), 2)
                gen_score = calculate_tanim_sim(s, mol)
                diff = abs(sim - gen_score)
                print("GT:", round(sim,2), "Gen:", gen_score, "diff:", round(diff,2), s, out )
                ground_truths.append(sim)
                gens.append(gen_score)
                diffs.append(diff)
            except:
                print(f"GT: {sim} {out}")
                invalids += 1
                ground_truths.append(sim)
                gens.append(gen_score)
                diffs.append(0)
        else:
            print(f"GT: {sim} {out}")
            invalids += 1
            ground_truths.append(sim)
            gens.append(gen_score)
            diffs.append(0)
        # except:
        #     print(f"GT: {sim} {out}")
        #     invalids += 1
        #     raise

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[16:38:00] DEPRECATION WARNING: please use MorganGenerator
[16:38:00] DEPRECATION WARNING: please use MorganGenerator
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GT: 0.2 Gen: 0.45054945054945056 diff: 0.25 CC(=O)Nc1ccc(C(=O)NN=Cc2cccc([N+](=O)[O-])c2OC(=O)c2cccc(C)c2)cc1 <|begin_of_text|>[SIMILAR]CC(=O)Nc1ccc(C(=O)NN=Cc2cccc([N+](=O)[O-])c2OC(=O)c2cccc(C)c2)cc1 0.20[/SIMILAR][QED]0.02[/QED][WEIGHT]1012.57[/WEIGHT][FORMULA]C56H76N8O10[/FORMULA][CLOGP]11.55[/CLOGP][RINGCOUNT]4[/RINGCOUNT][TPSA]245.62[/TPSA][SAS]3.72[/SAS][SMILES]CCCCCCCCCCCCCCCCCC(=O)Nc1ccc(C(=O)NN=Cc2cc([N+](=O)[O-])cc([N+](=O)[O-])c2OC(=O)c2ccc(NC(=O)CCCCCCCCCCCCCCCCC)cc2)cc1.N#C[O-].[C-]#[N+]C#N.[K+][/SMILES]


[16:38:01] DEPRECATION WARNING: please use MorganGenerator
[16:38:01] DEPRECATION WARNING: please use MorganGenerator
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GT: 0.25 Gen: 0.3404255319148936 diff: 0.09 CC(=O)Nc1ccc(C(=O)NN=Cc2cccc([N+](=O)[O-])c2OC(=O)c2cccc(C)c2)cc1 <|begin_of_text|>[SIMILAR]CC(=O)Nc1ccc(C(=O)NN=Cc2cccc([N+](=O)[O-])c2OC(=O)c2cccc(C)c2)cc1 0.25[/SIMILAR][QED]0.10[/QED][WEIGHT]494.22[/WEIGHT][CLOGP]4.81[/CLOGP][FORMULA]C26H30N4O6[/FORMULA][RINGCOUNT]3[/RINGCOUNT][TPSA]136.92[/TPSA][SAS]2.60[/SAS][SMILES]CCCCCCCCC(=O)NN=Cc1ccc(OC(=O)c2ccc3c(c2)CCC(=O)N3)c([N+](=O)[O-])c1[/SMILES]


...

[16:41:53] DEPRECATION WARNING: please use MorganGenerator
[16:41:53] DEPRECATION WARNING: please use MorganGenerator
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GT: 0.9 Gen: 1.0 diff: 0.1 CCOc1cc(OC)c(Br)cc1CC1CCCNC1 <|begin_of_text|>[SIMILAR]CCOc1cc(OC)c(Br)cc1CC1CCCNC1 0.90[/SIMILAR][WEIGHT]327.08[/WEIGHT][CLOGP]3.33[/CLOGP][FORMULA]C15H22BrNO2[/FORMULA][SAS]2.81[/SAS][SMILES]CCOc1cc(OC)c(Br)cc1CC1CCCNC1[/SMILES]


[16:41:54] DEPRECATION WARNING: please use MorganGenerator
[16:41:54] DEPRECATION WARNING: please use MorganGenerator
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GT: 0.95 Gen: 1.0 diff: 0.05 CCOc1cc(OC)c(Br)cc1CC1CCCNC1 <|begin_of_text|>[SIMILAR]CCOc1cc(OC)c(Br)cc1CC1CCCNC1 0.95[/SIMILAR][WEIGHT]327.08[/WEIGHT][CLOGP]3.38[/CLOGP][FORMULA]C15H22BrNO2[/FORMULA][SAS]2.80[/SAS][QED]0.90[/QED][SMILES]CCOc1cc(OC)c(Br)cc1CC1CCCNC1[/SMILES]
GT: 1.0 Gen: 1.0 diff: 0.0 CCOc1cc(OC)c(Br)cc1CC1CCCNC1 <|begin_of_text|>[SIMILAR]CCOc1cc(OC)c(Br)cc1CC1CCCNC1 1.00[/SIMILAR][SMILES]CCOc1cc(OC)c(Br)cc1CC1CCCNC1[/SMILES]


[16:41:54] DEPRECATION WARNING: please use MorganGenerator
[16:41:54] DEPRECATION WARNING: please use MorganGenerator


In [21]:
combined = list(zip(ground_truths, gens, diffs))
combined.sort(key=lambda x: x[0])
ground_truths, gens, diffs = zip(*combined)
d = np.mean(np.array(diffs).reshape(-1,N_mols),axis=1)
rmse = metrics.mean_squared_error(ground_truths, gens, squared=False)
mape = metrics.mean_absolute_percentage_error(ground_truths, gens)
correlation = correlation, pvalue = spearmanr(ground_truths, gens)
generate_plot(ground_truths, gens, d,'Similarity CG', rmse, mape, correlation, invalids, len(ground_truths), (0,1), 0)

/auto/home/menuab/miniforge3/envs/llama3/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
